## Functions

__Function for getting the dynamic parameters of the cell model__

In [83]:
def get_dyn_param(data,cell_temp):    
    
    ind = np.where(np.array(data['temps']) == cell_temp)[0][0]

    eta = data['etaParam'][ind]
    Q = data['QParam'][ind]
    gamma = data['GParam'][ind]
    M0 = data['M0Param'][ind]
    M = data['MParam'][ind]
    R0 = data['R0Param'][ind]
    RC = data['RCParam'][ind]
    RC = np.exp(-np.divide(1,RC))
    R = data['RParam'][ind]

    return (eta,Q,gamma,M0,M,R0,RC,R)

In [84]:
from scipy.interpolate import interp1d

def interp_ocv(z,OCV):
    
    dum = interp1d(z,OCV)
    
    return dum

__OCV calculation for any SOC value Function__

In [85]:
def OCV_from_SOC(z,temp,SOC,OCV_0,OCV_rel):

    # OCV calculation from the given SOC (Coloumb Counting)

    OCV_0_z = interp_ocv(SOC,OCV_0)(z)
    OCV_rel_z = interp_ocv(SOC,OCV_rel)(z)

    ocv_z = OCV_0_z + temp*OCV_rel_z
    
    # Setting the short-circuit cell voltage to zero
    # As there is no battery source at the resitor
    ocv_z[np.isnan(z)] = 0

    return (ocv_z)

## Main Code

__Importing the Dynamic model__

In [86]:
# import json
  
# # Opening JSON file
# f = open('modeldyn.json')
  
# # returns JSON object as 
# # a dictionary
# data = json.load(f)
  
# # Closing file
# f.close()

__Importing Dynamic Model__

In [87]:
data = pd.read_pickle('E2model.pkl')

__User Inputs__

In [88]:
# Number of cells in series in a module
Ns = 3

# Number of modules in parallel
Np = 3

# Simulation Time
max_time = 5

# Pack rests at time t0
t0 = 5

# Storage variable
z_pack = np.zeros((max_time,Ns,Np))
i_pack = np.zeros((max_time,Ns,Np))

# Initial state of the cells in the pack
z = 0.25*np.ones((Ns,Np))
irc = np.zeros((Ns,Np))
h = np.zeros((Ns,Np))

# Cell Interconnect resistance
R_inter = 125e-6

# Cell Temperature
# It is assumed that the cell temperature remains constant at 25°C
cell_temp = 25

# Random values initialization
random_init = 1

In [89]:
# Cell Parameters calculated from dynamic model

eta = get_dyn_param(data,cell_temp)[0]*np.ones((Ns,Np)) 
Q = get_dyn_param(data,cell_temp)[1]*np.ones((Ns,Np)) 
gamma = get_dyn_param(data,cell_temp)[2]*np.ones((Ns,Np)) 
M0 = get_dyn_param(data,cell_temp)[3]*np.ones((Ns,Np)) 
M = get_dyn_param(data,cell_temp)[4]*np.ones((Ns,Np)) 
R0 = get_dyn_param(data,cell_temp)[5]*np.ones((Ns,Np)) 
RC = get_dyn_param(data,cell_temp)[6]*np.ones((Ns,Np)) 
R = get_dyn_param(data,cell_temp)[7]*np.ones((Ns,Np)) 

In [90]:
SOC = data['SOC']
OCV_0  = data['OCV0']
OCV_rel = data['OCVrel']

In [91]:
# Fault parameters

#1.Open Cicuit fault
# Cell will have infinite resistance
R0_open = np.inf

#2.Short circuit fault
# During short circuit SOC cannot be measured
# We will assume some resistance for the cell

z_short = np.nan

R0_short = 0.0025

In [92]:
# if random_init:
    
#     z = 0.25 + 0.4*np.random.rand(Ns,Np)
#     Q = 4.5 + np.random.rand(Ns,Np)
#     R0 = 0.005 + 0.020*np.random.rand(Ns,Np)

In [93]:
# Updating the cell resistance with interconnect resistance

R0 = R0 + 2*R_inter

In [94]:
z[2][1] = np.nan

In [95]:
z


array([[0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25],
       [0.25,  nan, 0.25]])

__Simulation Parameters__

In [96]:
# Pack capcity is minimum module capacity

module_cap = np.sum(Q, axis = 1)
pack_cap = min(module_cap)

# Charge/Discharge current
# 10C charge/discharge

I = pack_cap*10

# # Introducing the faulty cell
# z[3][2] = z_short

In [97]:
z

array([[0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25],
       [0.25,  nan, 0.25]])

In [98]:
irc

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [99]:
for k in np.arange(0,max_time,1):
    
    
    # =========== Terminal Voltage Calculation =========== 

    v = OCV_from_SOC(z,cell_temp,SOC,OCV_0,OCV_rel)
    v = v + np.multiply(M,h) - np.multiply(R,irc)

    
    
# Author has not included this step. If we do not set the voltage to zero,
# Hysterisis ans Diffusion voltage terms are also added in the calculation
# It is to be confirmed in theory if the cell is cutoff or not
    
#     # Setting the short-circuit cell voltage to zero
#     # As there is no battery source at the resitor
#     v[np.isnan(v)] = 0


    
    # If the cell is short-ciruit then current is going to external resitor
    R0[np.isnan(z)] = R0_short
    
#     print('R0')
#     print(R0)
    
#     print('v')
#     print(v)


    # =========== Pack voltage calculation ===========

    # Module current calculation
    im = (np.sum(np.divide(v,R0),axis = 1) - I)
    
#     print('im')
#     print(im)

    # Module equivalent resistance reciprocal
    r_invm = np.sum(np.divide(1,R0),axis = 1)
    
    # Module voltage
    Vm = np.divide(im,r_invm)
    
#     print('Vm')
#     print(Vm)
    
    # Pack Voltage
    V_pack = sum(Vm)
    
#     print('Vdum')
#     print(np.tile(Vm,Np).reshape(Np,Ns).T)

    # Cell voltage calculation
    V_cell = v - np.tile(Vm,Np).reshape(Np,Ns).T
    
#     print('V_cell')
#     print(V_cell)

    # Cell current calculation
    i_cell = np.divide(V_cell,R0)
    
#     print('i_cell')
#     print(i_cell)

    # Cell new SOC calculation
    z = z-(1/3600)*(np.divide(i_cell,Q))
    
    # Short-circuiting the cell below discharge capacity
    z[z<0] = np.nan

    # Updating the capacitor volage
    irc = np.multiply(RC,irc) + np.multiply((1-RC),i_cell)


    # =========== Hysterisis calculation ===========

    fac = -np.abs(np.multiply(gamma,i_cell))
    fac = np.divide(fac,(3600*Q))
    fac = np.exp(fac)

    # Hysterisis voltage
    h = np.multiply(fac,h) + np.multiply((1-fac),np.sign(i_cell))


    # =========== Charging/Discharging Strategy =========== 

    if np.nanmin(z)<0.05:

        # Start charging
        I = -abs(I)
        print('Charging')

    elif np.nanmax(z)>0.95:

        # Start Discharging
        I = abs(I)
        print('Discharging')

    elif k>t0:

        # Cell Rests
        I = 0
        print('Rest')
        
#     print(z)

    print('v')
    print(v)
    print('R0')
    print(R0)
    print('Vm')
    print(Vm)
    print('Vcell')
    print(V_cell)
    print('icell')
    print(i_cell)
    print('z')
    print(z)
    print('irc')
    print(irc)
    print(np.multiply(RC,irc))
    print('RC')
    print(RC)
    print('fac')
    print(fac)
    print('h')
    print(h)
    print('\n\n')


v
[[3.82230172 3.82230172 3.82230172]
 [3.82230172 3.82230172 3.82230172]
 [3.82230172 0.         3.82230172]]
R0
[[0.01141324 0.01141324 0.01141324]
 [0.01141324 0.01141324 0.01141324]
 [0.01141324 0.0025     0.01141324]]
Vm
[3.23629621 3.23629621 0.8966218 ]
Vcell
[[ 0.58600551  0.58600551  0.58600551]
 [ 0.58600551  0.58600551  0.58600551]
 [ 2.92567992 -0.8966218   2.92567992]]
icell
[[  51.34436834   51.34436834   51.34436834]
 [  51.34436834   51.34436834   51.34436834]
 [ 256.34091236 -358.64871969  256.34091236]]
z
[[0.24722222 0.24722222 0.24722222]
 [0.24722222 0.24722222 0.24722222]
 [0.23613172        nan 0.23613172]]
irc
[[  17.43327889   17.43327889   17.43327889]
 [  17.43327889   17.43327889   17.43327889]
 [  87.0370551  -121.77427352   87.0370551 ]]
[[ 11.51404719  11.51404719  11.51404719]
 [ 11.51404719  11.51404719  11.51404719]
 [ 57.48481199 -80.42748242  57.48481199]]
RC
[[0.66046366 0.66046366 0.66046366]
 [0.66046366 0.66046366 0.66046366]
 [0.66046366 0.66046